# Query Ocean Sensitive Areas with your Asset Data
### A script that automates the process of querying the HUB Ocean Ocean Sensitive Area Data Product to return results based on your own asset data.

In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.56 requires cryptography<40.0.2,>=40.0.0, but you have cryptography 44.0.2 which is incompatible.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.


In [2]:
pip install h3 --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [1]:
from odp.client import OdpClient # The SDK
from shapely.geometry import box
import pandas as pd
import pydeck as pdk
import os
import h3
from ipywidgets import widgets
from IPython.display import display, clear_output

In [2]:
base_url = "https://odcat.dev.hubocean.io"
client = OdpClient(base_url=base_url)

## Extract lat and long from CSV and add Hex6 collumn
### Expects a csv file called "asset.csv" with required columns: "asset_id", "lat", "long"

In [3]:
def load_and_process_asset_data(file_path: str = "asset.csv", max_size_mb: int = 5, distance_km: int = 50) -> pd.DataFrame:
    """
    Loads asset data from a CSV file, verifies required columns (case insensitive),
    and adds an H3 index (resolution 6) column based on latitude and longitude.
    Also computes surrounding H3 indexes within a specified distance.
    """
    max_size_bytes = max_size_mb * 1024 * 1024  # Convert MB to Bytes
    file_size = os.path.getsize(file_path)
    if file_size > max_size_bytes:
        raise ValueError(f"Error: File size exceeds {max_size_mb}MB limit ({file_size / (1024*1024):.2f}MB).")
    
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, encoding="utf-8")
        elif file_path.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format. Use CSV or Excel (.xls/.xlsx).")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="ISO-8859-1")
        
    df.columns = df.columns.str.lower()
    lat_columns = {'latitude', 'lat'}
    lon_columns = {'longitude', 'long', 'lon'}
    
    lat_col = next((col for col in df.columns if col in lat_columns), None)
    lon_col = next((col for col in df.columns if col in lon_columns), None)
    
    if not lat_col or not lon_col:
        raise ValueError("Error: The file must contain 'latitude' and 'longitude' (or 'lat' and 'long') columns.")
    
    df['h3_index'] = df.apply(lambda row: h3.latlng_to_cell(row[lat_col], row[lon_col], 6), axis=1)
    
    def get_h3_neighbors(h3_index):
        num_rings = round(distance_km / 8.74)  # Approximate conversion for resolution 6
        return list(h3.grid_disk(h3_index, num_rings))
    
    df['h3_neighbors'] = df['h3_index'].apply(get_h3_neighbors)
    return df

def interactive_h3_processing():
    global df  # Ensure df is accessible globally
    file_path = "asset.csv"
    
    distance_slider = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=5,
        description='Distance (km):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style={'description_width': 'initial'}
    )
    
    process_button = widgets.Button(
        description='Process Data',
        button_style='primary',
        tooltip='Click to process the file',
        icon='check'
    )
    
    output = widgets.Output()
    
    def on_button_clicked(b):
        global df  # Ensure df is accessible globally
        with output:
            clear_output()
            try:
                print(f"Processing file: {file_path}")
                print(f"Distance for H3 neighbors: {distance_slider.value} km")
                
                df = load_and_process_asset_data(
                    file_path=file_path,
                    distance_km=distance_slider.value
                )
                
                print("\nProcessed data sample (first 5 rows):")
                display(df.head(5))
                
                print(f"\nTotal records processed: {len(df)}")
                avg_neighbors = df['h3_neighbors'].apply(len).mean()
                print(f"Average number of neighboring hexagons: {avg_neighbors:.2f}")
            
            except Exception as e:
                print(f"Error: {str(e)}")
    
    process_button.on_click(on_button_clicked)
    
    display(widgets.HTML(f"<h3>H3 Geospatial Data Processor</h3><p>File: <b>{file_path}</b></p>"))
    display(distance_slider)
    display(process_button)
    display(output)
    
    return {
        'distance': distance_slider,
        'output': output
    }

## Define your search area around assets

In [4]:
interactive_h3_processing()

HTML(value='<h3>H3 Geospatial Data Processor</h3><p>File: <b>asset.csv</b></p>')

IntSlider(value=50, continuous_update=False, description='Distance (km):', max=200, min=10, step=5, style=Slid…

Button(button_style='primary', description='Process Data', icon='check', style=ButtonStyle(), tooltip='Click t…

Output()

{'distance': IntSlider(value=50, continuous_update=False, description='Distance (km):', max=200, min=10, step=5, style=SliderStyle(description_width='initial')),
 'output': Output()}

## Query Ocean Sensitive Areas Dataset

In [5]:
# Create a mapping of asset_id to h3_index and its neighbors
asset_h3_mapping = []

for _, row in df.iterrows():
    asset_h3_mapping.append({
        'asset_id': row['asset_id'],
        'h3_index': row['h3_index'],
        'h3_neighbors': row['h3_neighbors']
    })

# Print first 5 entries for verification
print("Asset Locations with Neighbors:")
for entry in asset_h3_mapping[:5]:
    print(f"Asset ID: {entry['asset_id']}, H3 Index: {entry['h3_index']}")
    print(f"Neighbors: {entry['h3_neighbors'][:3]}... ({len(entry['h3_neighbors'])} total neighbors)\n")


Asset Locations with Neighbors:
Asset ID: 1, H3 Index: 864ce832fffffff
Neighbors: ['864ce832fffffff', '864ce8327ffffff', '864ce8307ffffff']... (61 total neighbors)

Asset ID: 2, H3 Index: 864cee4efffffff
Neighbors: ['864cee4efffffff', '864cee4e7ffffff', '864cee4c7ffffff']... (61 total neighbors)

Asset ID: 3, H3 Index: 8645a322fffffff
Neighbors: ['8645a322fffffff', '8645a3227ffffff', '8645a3207ffffff']... (61 total neighbors)

Asset ID: 4, H3 Index: 865e4b437ffffff
Neighbors: ['865e4b437ffffff', '865e4b5afffffff', '865e4b58fffffff']... (61 total neighbors)

Asset ID: 5, H3 Index: 864cd48dfffffff
Neighbors: ['864cd48dfffffff', '864cd48d7ffffff', '864cd412fffffff']... (61 total neighbors)



In [6]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("7af5bc0f-c12a-451b-9834-fa6c4e0a2c87"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

'Ocean Sensitive Areas - version 1.0.0 - table'

In [7]:
# Initialize an empty list to store the results for all assets
all_results = []

# Loop through each asset in the DataFrame
for _, row in df.iterrows():
    # Get the asset's H3 index and neighbors
    hex6_set = {row['h3_index']}  # Add the asset's H3 index
    hex6_set.update(row['h3_neighbors'])  # Add its neighbors

    # Convert the set to a list for query construction
    hex6_array = list(hex6_set)

    # Build the query using OR conditions
    query = " OR ".join(f'hex6 == "{h}"' for h in hex6_array)

    # Execute the query to get results for the current asset and its neighbors
    result_df = next(osa_data.select(query).dataframes())

    # Add a column to indicate whether the record is a neighbor or the main asset
    result_df['is_neighbor'] = result_df['hex6'].apply(
        lambda x: 'Neighbor' if x != row['h3_index'] and x in row['h3_neighbors'] else 'Asset'
    )

    # Add the asset_id to the results (the current asset's ID)
    result_df['asset_id'] = row['asset_id']

    # Append the result to the all_results list
    all_results.append(result_df)

# Concatenate all the individual results into a single DataFrame
final_results_df = pd.concat(all_results, ignore_index=True)

## Results printed as a table and saved as csv

In [8]:
# Function to save the DataFrame as a CSV
def save_to_csv(df, filename="osa_results.csv"):
    try:
        df.to_csv(filename, index=False)
        print(f"Data successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving data: {e}")

# Assuming final_results_df is already created
print(f"Total rows in final result: {len(final_results_df)}")
# print(final_results_df.head(5))

# Save the DataFrame to a CSV file
save_to_csv(final_results_df)


Total rows in final result: 398
Data successfully saved to osa_results.csv


In [9]:
final_results_df.head(5)

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw,is_neighbor,asset_id
0,4.342,0.018,0.0,864ce8047ffffff,0.0,0.0,1.0,"POLYGON ((-64.8837 18.2467, -64.8598 18.2681, ...",1.0,42.946,0.542,0.424,Neighbor,1
1,4.092,0.023,0.0,864ce804fffffff,0.0,0.0,0.0,"POLYGON ((-64.8515 18.2125, -64.8276 18.2339, ...",0.0,9.432,0.511,0.085,Neighbor,1
2,4.251,0.023,1.0,864ce8067ffffff,0.0,0.0,1.0,"POLYGON ((-64.864 18.2959, -64.84 18.3172, -64...",1.0,59.259,0.531,0.588,Neighbor,1
3,4.178,0.025,0.0,864ce806fffffff,0.0,0.0,1.0,"POLYGON ((-64.8317 18.2617, -64.8078 18.2831, ...",1.0,42.608,0.522,0.420,Neighbor,1
4,2.948,0.074,0.0,864ce814fffffff,0.0,0.0,1.0,"POLYGON ((-64.8442 18.345, -64.8203 18.3664, -...",1.0,40.073,0.368,0.395,Neighbor,1


In [11]:
def categorize_shannon(shannon):
    if shannon < 2.0:
        return "Low Biodiversity"
    elif 2.0 <= shannon <= 4.0:
        return "Medium Biodiversity"
    else:
        return "High Biodiversity"

def categorize_simpson(simpson):
    if simpson > 0.5:
        return "High Dominance, Low Evenness"
    elif 0.2 <= simpson <= 0.5:
        return "Moderate Dominance"
    else:
        return "Low Dominance, High Evenness"

def generate_asset_report(df):
    report_list = []
    
    # Compute asset ranking based on average Shannon Index
    asset_ranks = df.groupby("asset_id")["shannon"].mean().rank(ascending=False).to_dict()
    df["biodiversity_rank"] = df["asset_id"].map(asset_ranks)
    
    # Sort assets by biodiversity rank (higher Shannon Index first)
    sorted_assets = df.groupby("asset_id")["shannon"].mean().sort_values(ascending=False).index
    
    for asset_id in sorted_assets:
        asset_df = df[df['asset_id'] == asset_id]
        
        # Extract exact location row
        exact_location = asset_df[asset_df['is_neighbor'].str.lower() == "exact"]
        if not exact_location.empty:
            exact_shannon = exact_location['shannon'].values[0]
            exact_simpson = exact_location['simpson'].values[0]
            ecosystems = [eco.capitalize() for eco in ['mangrove', 'seamount', 'cold_water_coral', 'seagrass', 'coral'] 
                          if exact_location[eco].values[0] > 0]
        else:
            exact_shannon, exact_simpson, ecosystems = None, None, []

        # Extract surrounding data
        neighbors = asset_df[asset_df['is_neighbor'].str.lower() == "neighbor"]
        avg_shannon = neighbors['shannon'].mean() if not neighbors.empty else None
        avg_simpson = neighbors['simpson'].mean() if not neighbors.empty else None

        # Rank and total assets
        asset_rank = asset_ranks.get(asset_id, None)
        total_assets = df['asset_id'].nunique()
        
        # Helper function to format with default value
        def safe_format(value):
            return f"{value:.3f}" if value is not None else "N/A"
        
        # Construct Report
        report = f"""
Biodiversity Rank: #{int(asset_rank) if asset_rank else "N/A"} out of {total_assets}
Asset ID: {asset_id}
-----------------------------------
Exact Location:
  - Shannon Index: {safe_format(exact_shannon)} ({categorize_shannon(exact_shannon) if exact_shannon is not None else "N/A"})
  - Simpson Index: {safe_format(exact_simpson)} ({categorize_simpson(exact_simpson) if exact_simpson is not None else "N/A"})
  - Ecosystems: {', '.join(ecosystems) if ecosystems else "None"}

Surrounding Area (50km):
  - Avg Shannon Index: {safe_format(avg_shannon)} ({categorize_shannon(avg_shannon) if avg_shannon is not None else "N/A"})
  - Avg Simpson Index: {safe_format(avg_simpson)} ({categorize_simpson(avg_simpson) if avg_simpson is not None else "N/A"})
  - % Coverage: 
    - Coral: {safe_format(neighbors['coral'].mean() * 100 if not neighbors.empty else None)}%
    - Seagrass: {safe_format(neighbors['seagrass'].mean() * 100 if not neighbors.empty else None)}%
    - Cold Water Coral: {safe_format(neighbors['cold_water_coral'].mean() * 100 if not neighbors.empty else None)}%
"""
        report_list.append(report)
    
    return report_list

# Run the function and print reports
reports = generate_asset_report(final_results_df)
for r in reports:
    print(r)


Biodiversity Rank: #1 out of 20
Asset ID: 6
-----------------------------------
Exact Location:
  - Shannon Index: N/A (N/A)
  - Simpson Index: N/A (N/A)
  - Ecosystems: None

Surrounding Area (50km):
  - Avg Shannon Index: 2.931 (Medium Biodiversity)
  - Avg Simpson Index: 0.178 (Low Dominance, High Evenness)
  - % Coverage: 
    - Coral: 76.000%
    - Seagrass: 62.000%
    - Cold Water Coral: 0.000%


Biodiversity Rank: #2 out of 20
Asset ID: 3
-----------------------------------
Exact Location:
  - Shannon Index: N/A (N/A)
  - Simpson Index: N/A (N/A)
  - Ecosystems: None

Surrounding Area (50km):
  - Avg Shannon Index: 2.406 (Medium Biodiversity)
  - Avg Simpson Index: 0.204 (Moderate Dominance)
  - % Coverage: 
    - Coral: 86.957%
    - Seagrass: 86.957%
    - Cold Water Coral: 56.522%


Biodiversity Rank: #2 out of 20
Asset ID: 17
-----------------------------------
Exact Location:
  - Shannon Index: N/A (N/A)
  - Simpson Index: N/A (N/A)
  - Ecosystems: None

Surrounding Area 

## Display on Map (Experimental)

In [116]:
def create_h3_pydeck_map(df, hex_column, value_column, scale_factor=500, center_lat=58.0, center_lon=10.0, zoom=5):
    return pdk.Deck(
        layers=[pdk.Layer( 'H3HexagonLayer', df, get_hexagon=hex_column, get_fill_color=f'[254, 119, 76, {scale_factor} * {value_column}]', opacity=1, extruded=False,
            get_line_color=[32, 10, 58, 10], line_width_min_pixels=1, pickable=True, filled=True, stroked=True )],
        initial_view_state=pdk.ViewState(latitude=center_lat, longitude=center_lon, zoom=zoom, pitch=0), map_provider="mapbox", map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'})

In [117]:
# Bounding box for Caribbean
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 18
lat_max = 21
lon_min = -77
lon_max = -72
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((-72 18, -72 21, -77 21, -77 18, -72 18))'

In [119]:
# Vizualize data sample for the Shannon index. You can swap out shannon_norm for any other attribute e.g. seagrass
map_viz = create_h3_pydeck_map(final_results_df, 'hex6', 'coral', scale_factor=500, center_lat=((lat_min + lat_max) / 2), center_lon=((lon_min + lon_max) / 2), zoom=6)
map_viz

{
  "initialViewState": {
    "latitude": 19.5,
    "longitude": -74.5,
    "pitch": 0,
    "zoom": 6
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "S": 40.196,
          "S_raw": 0.396,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.9679 18.2659, -64.944 18.2872, -64.9482 18.315, -64.9762 18.3214, -65.0001 18.3, -64.996 18.2722, -64.9679 18.2659))",
          "hex6": "864ce800fffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 3.007,
          "shannon_norm": 0.376,
          "simpson": 0.063
        },
        {
          "S": 43.831,
          "S_raw": 0.433,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.9482 18.315, -64.9243 18.3364, -64.9284 18.3641, -64.9565 18.3704, -64.9804 18.3491, -64.9762 18.3214, -64.9482 18.315))",
          "hex6": "864ce802fffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 4.771,
          "shannon_norm": 0.596,
          "simpson": 0.019
        },
        {
          "S": 42.946,
          "S_raw": 0.424,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.8837 18.2467, -64.8598 18.2681, -64.864 18.2959, -64.8921 18.3022, -64.916 18.2809, -64.9118 18.2531, -64.8837 18.2467))",
          "hex6": "864ce8047ffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 4.342,
          "shannon_norm": 0.542,
          "simpson": 0.018
        },
        {
          "S": 9.432,
          "S_raw": 0.085,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 0.0,
          "geometry": "POLYGON ((-64.8515 18.2125, -64.8276 18.2339, -64.8317 18.2617, -64.8598 18.2681, -64.8837 18.2467, -64.8796 18.2189, -64.8515 18.2125))",
          "hex6": "864ce804fffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 4.092,
          "shannon_norm": 0.511,
          "simpson": 0.023
        },
        {
          "S": 26.133,
          "S_raw": 0.254,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.9357 18.2317, -64.9118 18.2531, -64.916 18.2809, -64.944 18.2872, -64.9679 18.2659, -64.9638 18.2381, -64.9357 18.2317))",
          "hex6": "864ce8057ffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 0.0,
          "seamount": 0.0,
          "shannon": 4.19,
          "shannon_norm": 0.523,
          "simpson": 0.021
        },
        {
          "S": 59.259,
          "S_raw": 0.588,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.864 18.2959, -64.84 18.3172, -64.8442 18.345, -64.8723 18.3514, -64.8962 18.33, -64.8921 18.3022, -64.864 18.2959))",
          "hex6": "864ce8067ffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 1.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 4.251,
          "shannon_norm": 0.531,
          "simpson": 0.023
        },
        {
          "S": 42.608,
          "S_raw": 0.42,
          "asset_id": 1,
          "cold_water_coral": 0.0,
          "coral": 1.0,
          "geometry": "POLYGON ((-64.8317 18.2617, -64.8078 18.2831, -64.8119 18.3108, -64.84 18.3172, -64.864 18.2959, -64.8598 18.2681, -64.8317 18.2617))",
          "hex6": "864ce806fffffff",
          "is_neighbor": "Neighbor",
          "mangrove": 0.0,
          "seagrass": 1.0,
          "seamount": 0.0,
          "shannon": 4.178,
      